In [ ]:
import sys
import os
from pprint import pprint


current_dir = os.path.dirname(os.path.realpath(os.getcwd()))
different_path = os.path.abspath(os.path.join(current_dir, '../'))
sys.path.append(current_dir)

from modules.graphs import build_graph, graph_names
from modules.query_resolver import query_resolver
from modules.queries import queries

In [ ]:
import psycopg2
import psycopg2.extras

from urllib.parse import urlparse
import os
from decouple import AutoConfig
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
config = AutoConfig(search_path=parent_directory)
url = urlparse(config("db_url"))
connection = psycopg2.connect(
    host=url.hostname,
    port=url.port,
    database=url.path[1:],
    user=url.username,
    password=url.password
)
cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)

In [ ]:
result = query_resolver(cursor, queries, "list_guilds")
guild_id = result.iloc[0]["guild_id"]
# print("list_guilds")
print(f"guild_id = {guild_id}")
# pprint(result)
assert result.shape[1] == 3

In [ ]:
fig = build_graph(
    cursor,
    "user_longest_avg_msg_length",
    {
        "guild_id" : guild_id
    }
)
fig["fig"].update_layout(fig["layout"])

In [ ]:
fig = build_graph(
    cursor,
    "guild_author_most_messages",
    {
        "guild_id" : guild_id
    }
)
fig["fig"].update_layout(fig["layout"])

In [ ]:
fig = build_graph(
    cursor,
    "guild_author_most_days_with_messages",
    {
        "guild_id" : guild_id
    }
)
fig["fig"].update_layout(fig["layout"])

In [ ]:
fig = build_graph(
    cursor,
    "guild_author_most_reactions",
    {
        "guild_id" : guild_id
    }
)
fig["fig"].update_layout(fig["layout"])

In [ ]:
fig = build_graph(
    cursor,
    "guild_author_most_reactions",
    {
        "guild_id" : guild_id
    }
)
fig["fig"].update_layout(fig["layout"])

In [ ]:
fig = build_graph(
    cursor,
    "guild_author_most_attachments",
    {
        "guild_id" : guild_id
    }
)
fig["fig"].update_layout(fig["layout"])

In [ ]:
fig = build_graph(
    cursor,
    "guild_author_most_attachments",
    {
        "guild_id" : guild_id
    }
)
fig["fig"].update_layout(fig["layout"])

In [ ]:
for graph_name in graph_names.keys():
    fig = build_graph(
        cursor,
        "guild_author_most_attachments",
        {
            "guild_id" : guild_id
        }
    )
    fig["fig"].update_layout(fig["layout"])